In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Github/AIPlus99/HW6

/content/drive/MyDrive/Colab Notebooks/Github/AIPlus99/HW6


In [3]:
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation

In [4]:
# PPO-based training loop for CodeGenerator
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
from torch import nn
from copy import deepcopy
from tqdm import tqdm
from code_generator_1_3b import CodeGenerator
from code_discriminator import CodeDiscriminator
from requirement_to_code_dataset import get_train_dataloader
from value_head import ValueHead
from bitsandbytes.optim import AdamW8bit
import gc

def batched_discriminator(discriminator, texts, batch_size=1):
    preds = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        logits = discriminator(chunk)
        preds.append(logits.cpu())
    return torch.cat(preds, dim=0).to(discriminator.classifier[0].weight.device)

FORMAT_PENALTY_WEIGHT_H = 1.0 / 8.0
FORMAT_PENALTY_WEIGHT_CPP = 1.0 / 8.0

def compute_format_penalties_h(header_texts):
    raw_penalties = torch.tensor([
        sum([
            -1.0 if not text.strip().startswith("```cpp") else 0.0,
            -2.0 if not text.strip().endswith("```") else 0.0,
            -1.0 if ("UCLASS" not in text and "USTRUCT" not in text) else 0.0,
            -1.0 if "#pragma once" not in text else 0.0,
            -1.0 if "#include \"CoreMinimal.h\"" not in text else 0.0,
            -1.0 if '.generated.h' not in text else 0.0,
            -1.0 if 'GENERATED_BODY()' not in text else 0.0,
        ])
        for text in header_texts
    ], dtype=torch.float32)
    return FORMAT_PENALTY_WEIGHT_H * raw_penalties

def compute_format_penalties_cpp(cpp_texts):
    raw_penalties = torch.tensor([
        sum([
            -1.0 if not text.strip().startswith("```cpp") else 0.0,
            -2.0 if not text.strip().endswith("```") else 0.0,
            -5.0 if "::" not in text else 0.0,
        ])
        for text in cpp_texts
    ], dtype=torch.float32)
    return FORMAT_PENALTY_WEIGHT_CPP * raw_penalties

def compute_log_ratios(log_probs, ref_log_probs, clip=10):
    log_ratio = (log_probs - ref_log_probs).clamp(min=-clip, max=clip)
    return torch.exp(log_ratio)

def compute_advantages(rewards, values, normalize=False):
    adv = rewards - values.detach()
    if normalize:
        return (adv - adv.mean()) / (adv.std() + 1e-6)
    else:
        return adv

def compute_ppo_loss(ratios, advantages, clip_eps):
    ratios = ratios.squeeze()
    advantages = advantages.squeeze()
    clipped = torch.clamp(ratios, 1 - clip_eps, 1 + clip_eps)

    surrogate1 = ratios * advantages
    surrogate2 = clipped * advantages

    surrogate = torch.where(
        advantages >= 0,
        torch.min(surrogate1, surrogate2),
        torch.max(surrogate1, surrogate2)
    )

    return -torch.mean(surrogate)

def compute_value_loss(values_disc, reward_disc, values_format, reward_format):
    loss_disc = nn.functional.mse_loss(values_disc, reward_disc.to(values_disc.dtype))
    loss_format = nn.functional.mse_loss(values_format, reward_format.to(values_format.dtype))
    return loss_disc + loss_format

def compute_sft_loss(generator, tokenizer, prompts, targets):
    joined_inputs = [p + t for p, t in zip(prompts, targets)]
    inputs = tokenizer(joined_inputs, return_tensors="pt", padding=True, truncation=True,
                       max_length=generator.max_length).to(generator.device)
    labels = inputs["input_ids"].clone()

    prompt_inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True,
                              max_length=generator.max_length)
    prompt_lens = [len(p[p != tokenizer.pad_token_id]) for p in prompt_inputs["input_ids"]]
    for i, l in enumerate(prompt_lens):
        labels[i, :l] = -100

    output = generator(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    return output.loss


def update_generator(total_loss, generator, g_optimizer):
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
    g_optimizer.step()
    g_optimizer.zero_grad()

def update_discriminator(discriminator, d_optimizer, reference_texts, generated_texts, device):
    all_texts = reference_texts + generated_texts
    labels = torch.tensor([1] * len(reference_texts) + [0] * len(generated_texts), dtype=torch.float32).to(device)
    preds = batched_discriminator(discriminator, all_texts)
    loss = nn.BCEWithLogitsLoss()(preds, labels)

    d_optimizer.zero_grad()
    loss.backward()
    d_optimizer.step()


def ppo_train_separated(h_generator, cpp_generator, train_loader, epochs=3, lr=2e-5, clip_eps=0.2):
    h_generator.train()
    cpp_generator.train()

    h_discriminator = CodeDiscriminator().to(device)

    d_path = os.path.join("./checkpoint_1_3b", "h_discriminator.pt")
    if os.path.exists(d_path):
        h_discriminator.load_state_dict(torch.load(d_path, map_location=device))
        print("H Discriminator loaded.")
    else:
        print("Training H discriminator from scratch.")


    cpp_discriminator = CodeDiscriminator().to(device)

    d_path = os.path.join("./checkpoint_1_3b", "cpp_discriminator.pt")
    if os.path.exists(d_path):
        cpp_discriminator.load_state_dict(torch.load(d_path, map_location=device))
        print("CPP Discriminator loaded.")
    else:
        print("Training CPP discriminator from scratch.")

    h_discriminator.train()
    cpp_discriminator.train()

    h_ref_generator = CodeGenerator()
    cpp_ref_generator = CodeGenerator()

    h_ref_generator = deepcopy(h_generator)
    h_ref_generator.eval()
    for param in h_ref_generator.parameters():
        param.requires_grad = False

    cpp_ref_generator = deepcopy(cpp_generator)
    cpp_ref_generator.eval()
    for param in cpp_ref_generator.parameters():
        param.requires_grad = False

    h_g_optimizer = AdamW8bit(h_generator.parameters(), lr=lr)
    cpp_g_optimizer = AdamW8bit(cpp_generator.parameters(), lr=lr)

    h_d_optimizer = AdamW8bit(h_discriminator.parameters(), lr=1e-4)
    cpp_d_optimizer = AdamW8bit(cpp_discriminator.parameters(), lr=1e-4)

    os.makedirs("./checkpoint_1_3b", exist_ok=True)

    sft_coef = 0.05
    value_coef = 0.5

    do_SFT_h = False
    do_SFT_cpp = False

    for epoch in range(epochs):
        loop = tqdm(train_loader)

        for step, batch in enumerate(loop):
            torch.cuda.empty_cache()

            requirements = batch["requirement"]
            reference_headers = batch["header_code"]
            reference_cpps = batch["cpp_code"]

            # === [Phase 1] Header PPO Sampling ===

            header_prompts = [h_generator.create_short_prompt_header(r) for r in requirements]

            # === [Phase 1-A] SFT Loss & Step ===
            if do_SFT_h:
                sft_loss = compute_sft_loss(h_generator, h_generator.tokenizer, header_prompts, reference_headers)
                print(f"\nsft_loss : {sft_loss}")
                sft_loss = sft_loss * sft_coef
                h_g_optimizer.zero_grad()
                sft_loss.backward()
                h_g_optimizer.step()

                # Cleanup SFT memory
                sft_loss = sft_loss.detach()
                del sft_loss
                torch.cuda.empty_cache()
                gc.collect()

            # === [Phase 1-B] PPO Loss ===
            header_output = h_generator.sample_header_with_partial_grad(requirements, max_track_tokens=80)
            print("\n===== .h =====")
            print(header_output["header_texts"][0])
            print(header_output["header_texts"][1])

            header_texts = header_output["header_texts"]

            with torch.no_grad():
                ref_header_output = h_ref_generator.sample_header_with_partial_grad(requirements, max_track_tokens=80)

            with torch.no_grad():
                d_scores = torch.sigmoid(h_discriminator(header_texts)).squeeze()
                reward_disc = 2 * (d_scores - 0.5)
                format_penalties = compute_format_penalties_h(header_texts).to(h_generator.device)
                if format_penalties.mean().item() < -0.01:
                    print("format_penalties < -0.01")
                    do_SFT_h = True
                else:
                    do_SFT_h = False
                header_rewards = reward_disc + format_penalties

            log_probs     = header_output["header_log_probs"].mean(dim=1)
            ref_log_probs = ref_header_output["header_log_probs"].mean(dim=1)
            ratios        = compute_log_ratios(log_probs, ref_log_probs)

            values_total, values_disc, values_format = h_generator.compute_value(header_prompts, header_texts, mode="h")
            advantages = compute_advantages(header_rewards, values_total)

            print(f"reward_disc : {reward_disc}")
            print(f"values_disc : {values_disc}")

            print(f"format_penalties : {format_penalties}")
            print(f"values_format : {values_format}")

            print(f"ratios : {ratios}")
            print(f"advantages : {advantages}")
            ppo_loss   = compute_ppo_loss(ratios, advantages, clip_eps)
            value_loss = compute_value_loss(values_disc, reward_disc, values_format, format_penalties)
            total_loss = ppo_loss + value_coef * value_loss

            # PPO update
            h_g_optimizer.zero_grad()
            total_loss.backward()
            h_g_optimizer.step()

            # === Discriminator training ===
            if reward_disc.mean().item() > -0.4 or (epoch == 0 and step < -1):
                print(f"\n[Step {step}] Training header discriminator...")
                update_discriminator(h_discriminator, h_d_optimizer, reference_headers, header_texts, h_generator.device)

            # === Logging ===
            loop.set_description(f"[Epoch {epoch+1}] Header")
            loop.set_postfix({
                "ppo":    ppo_loss.item(),
                "value":  value_loss.item(),
                "reward": header_rewards.mean().item()
            })

            # === Cleanup ===
            for var_name in [
                "log_probs", "ref_log_probs", "ratios",
                "values_total", "values_disc", "values_format",
                "d_scores", "reward_disc", "format_penalties", "header_rewards",
                "advantages", "ppo_loss", "value_loss", "total_loss"
            ]:
                var = locals().get(var_name)
                if isinstance(var, torch.Tensor):
                    locals()[var_name] = var.detach()
                del var

            del header_output, ref_header_output, header_texts, header_prompts
            torch.cuda.empty_cache()


            # === [Phase 2] CPP PPO 학습 ===

            cpp_prompts = [cpp_generator.create_short_prompt_cpp(r, h) for r, h in zip(requirements, reference_headers)]

            # === [Phase 2-A] SFT Loss & Step ===
            if do_SFT_cpp:
                sft_loss = compute_sft_loss(cpp_generator, cpp_generator.tokenizer, cpp_prompts, reference_cpps)
                print(f"\nsft_loss : {sft_loss}")
                sft_loss = sft_loss * sft_coef
                cpp_g_optimizer.zero_grad()
                sft_loss.backward()
                cpp_g_optimizer.step()

                # Clean up SFT memory
                sft_loss = sft_loss.detach()
                del sft_loss
                torch.cuda.empty_cache()
                gc.collect()

            # === [Phase 2-B] PPO + Value Loss ===

            cpp_output = cpp_generator.sample_cpp_with_partial_grad(requirements, reference_headers, max_track_tokens=80)
            print("\n===== .cpp =====")
            print(cpp_output["cpp_texts"][0])
            print(cpp_output["cpp_texts"][1])

            cpp_texts = cpp_output["cpp_texts"]

            with torch.no_grad():
                ref_cpp_output = cpp_ref_generator.sample_cpp_with_partial_grad(requirements, reference_headers, max_track_tokens=80)

            with torch.no_grad():
                d_scores = torch.sigmoid(cpp_discriminator(cpp_texts)).squeeze()
                reward_disc = 2 * (d_scores - 0.5)
                format_penalties = compute_format_penalties_cpp(cpp_texts).to(cpp_generator.device)
                if format_penalties.mean().item() < -0.01:
                    print("format_penalties < -0.01")
                    do_SFT_cpp = True
                else:
                    do_SFT_cpp = False
                cpp_rewards = reward_disc + format_penalties

            log_probs     = cpp_output["cpp_log_probs"].mean(dim=1)
            ref_log_probs = ref_cpp_output["cpp_log_probs"].mean(dim=1)
            ratios        = compute_log_ratios(log_probs, ref_log_probs)

            values_total, values_disc, values_format = cpp_generator.compute_value(cpp_prompts, cpp_texts, mode="cpp")
            advantages = compute_advantages(cpp_rewards, values_total)

            print(f"reward_disc : {reward_disc}")
            print(f"values_disc : {values_disc}")

            print(f"format_penalties : {format_penalties}")
            print(f"values_format : {values_format}")

            print(f"ratios : {ratios}")
            print(f"advantages : {advantages}")
            ppo_loss   = compute_ppo_loss(ratios, advantages, clip_eps)
            value_loss = compute_value_loss(values_disc, reward_disc, values_format, format_penalties)
            total_loss = ppo_loss + value_coef * value_loss

            # PPO step
            cpp_g_optimizer.zero_grad()
            total_loss.backward()
            cpp_g_optimizer.step()

            # === Train Discriminator on CPP ===
            if reward_disc.mean().item() > -0.4 or (epoch == 0 and step < -1):
                print(f"\n[Step {step}] Training cpp discriminator...")
                update_discriminator(cpp_discriminator, cpp_d_optimizer, reference_cpps, cpp_texts, cpp_generator.device)

            # === Logging ===
            loop.set_description(f"[Epoch {epoch+1}] CPP")
            loop.set_postfix({
                "ppo":    ppo_loss.item(),
                "value":  value_loss.item(),
                "reward": cpp_rewards.mean().item()
            })

            # === Cleanup ===
            for var_name in [
                "log_probs", "ref_log_probs", "ratios",
                "values_total", "values_disc", "values_format",
                "d_scores", "reward_disc", "format_penalties", "cpp_rewards",
                "advantages", "ppo_loss", "value_loss", "total_loss"
            ]:
                var = locals().get(var_name)
                if isinstance(var, torch.Tensor):
                    locals()[var_name] = var.detach()
                del var

            del cpp_output, ref_cpp_output, cpp_texts, cpp_prompts
            torch.cuda.empty_cache()


            # === 주기적 ref_generator 동기화 ===
            if step % 10 == 0:
                print(f"\n[Step {step}] Ref generator updated.")
                del h_ref_generator
                del cpp_ref_generator

                h_ref_generator = deepcopy(h_generator)
                h_ref_generator.eval()
                for p in h_ref_generator.parameters():
                    p.requires_grad = False

                cpp_ref_generator = deepcopy(cpp_generator)
                cpp_ref_generator.eval()
                for p in cpp_ref_generator.parameters():
                    p.requires_grad = False

                # Save checkpoint
                h_generator.save("./checkpoint_1_3b/h_generator")
                cpp_generator.save("./checkpoint_1_3b/cpp_generator")
                torch.save(h_discriminator.state_dict(), os.path.join("./checkpoint_1_3b", "h_discriminator.pt"))
                torch.save(cpp_discriminator.state_dict(), os.path.join("./checkpoint_1_3b", "cpp_discriminator.pt"))

        with torch.no_grad():
            h_generated_responses = h_generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],max_track_tokens=0)
            cpp_generated_responses = cpp_generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"],max_track_tokens=0)
        print("\n===== .h =====")
        print(h_generated_responses["header_texts"][0])
        print("\n===== .cpp =====")
        print(cpp_generated_responses["cpp_texts"][0])


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_path = "./checkpoint_1_3b"

    generator = CodeGenerator(load_path="./checkpoint_1_3b/h_generator")
    # cpp_generator = CodeGenerator(load_path="./checkpoint_1_3b/cpp_generator")

    """
    with torch.no_grad():
        h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],  max_track_tokens=0)
        cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"], max_track_tokens=0)
    print("\n===== .h =====")
    print(h_generated_responses["header_texts"][0])
    print("\n===== .cpp =====")
    print(cpp_generated_responses["cpp_texts"][0])
    """
    train_loader = get_train_dataloader("unreal_code_dataset0.jsonl", batch_size=2, shuffle=True, limit= 90)

    ppo_train_separated(generator, generator, train_loader, epochs=40)



Loading model + LoRA from ./checkpoint_1_3b/h_generator


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Freezing all parameters except LoRA blocks...
Trainable params: 7,495,680 / 1,353,967,616 (0.55%)
Loaded value head.
H Discriminator loaded.
CPP Discriminator loaded.
Initializing new model from base: deepseek-ai/deepseek-coder-1.3b-instruct
Applying LoRA adaptation...
Freezing all parameters except LoRA blocks...
Trainable params: 7,495,680 / 1,353,967,616 (0.55%)
No value head to load
Initializing new model from base: deepseek-ai/deepseek-coder-1.3b-instruct
Applying LoRA adaptation...
Freezing all parameters except LoRA blocks...
Trainable params: 7,495,680 / 1,353,967,616 (0.55%)
No value head to load


  0%|          | 0/45 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.



===== .h =====
```cpp
#include "GameFramework/Actor.h"
#include "Engine/World.h"
#include "Components/StaticMeshComponent.h"
#include "GameFramework/Character.h"

UCLASS()
class ACharacterWall : public ACharacter
{
    GENERATED_BODY()

public:
    ACharacterWall();

    void BeginPlay() override;

    void SetupPlayerInputComponent(UInputComponent* PlayerInputComponent) override;

    void Jump() { bCanJump = false; }

private:
    UFUNCTION()
    void OnBeginOverlap(UPrimitiveComponent* OverlappedComp, AActor* OtherActor,
                        Types::FName Socket, int32 Tag, bool& bShouldCollide);

    UPROPERTY(EditAnywhere)
    float JumpHeight;

    UPROPERTY(EditAnywhere)
    float GroundDistance;

    bool bCanJump;
};
```
```cpp
#pragma once

UCLASS()
class AGravitySwitch : public AActor
{
    GENERATED_BODY()

public:
    AGravitySwitch();

    void Activate();

    void Deactivate();

protected:
    void BeginPlay() override;

    void EndPlay(const EEndPlayReason::Type En

...


sft_loss : 1.9595857858657837

===== .h =====
Some simple health display using some UPROPERTY() and some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some some

[Epoch 1] Header:  84%|████████▍ | 38/45 [4:07:09<1:15:35, 647.87s/it, ppo=0.156, value=0.0143, reward=-1.71]


sft_loss : 1.8489912748336792


[Epoch 1] Header:  84%|████████▍ | 38/45 [4:08:03<45:41, 391.68s/it, ppo=0.156, value=0.0143, reward=-1.71]  


KeyboardInterrupt: 

In [ ]:
generator.save("./checkpoint_1_3b/h_generator")
generator.save("./checkpoint_1_3b/cpp_generator")